In [1]:
backend = 'local' # If preferred, change this to one of the backends in your ~/.sjef/molpro/backends.xml that is ssh-accessible

In [2]:
methods = ['B3LYP','HF','MP2','CCSD','CCSD(T)']
bases = ['cc-pVDZ','cc-pVTZ']

[doi:10.1063/1.1357225](https://doi.org/10.1063/1.1357225) and [doi:10.1063/1.481544](https://doi.org/10.1063/1.481544)

In [3]:
molecules={}
molecules["F2"] = {"De_exp" : 163.35, "De_calc" : 161.04, "geometry" : "F;F,F,1.4113", "stoichiometry": "FF"}
molecules["H2"] = {"De_exp" : 458.04, "De_calc" : 458.13, "geometry" : "H;H,H,.7419", "stoichiometry": "HH"}

In [4]:
closed_shell_methods={'B3LYP' : "ks, b3lyp", "HF" : "rhf", "MP2":"mp2","MP3":"mp3","MP4":"mp4","CCSD":"ccsd","CCSD(T)":"ccsd(t)"}
open_shell_methods={'B3LYP' : "rks, b3lyp", "HF" : "rhf", "MP2":"rmp2","MP3":"rmp3","MP4":"rmp4","CCSD":"rccsd","CCSD(T)":"rccsd(t)"}


In [5]:
from pysjef import all_completed, DirectoryNode
from pysjef_molpro import no_errors, Project
import numpy

In [6]:
atoms={}
for molecule in molecules:
    for atom in molecules[molecule]["stoichiometry"]:
        atoms[atom]={"geometry" : atom, "open_shell" : True}

In [7]:
root = DirectoryNode('atomisation')

In [8]:
def proj(molecule,method,basis):
    return Project((molecule+":"+method+":"+basis).replace('(T)','-T'),location='atomisation',suffix='molpro')

In [9]:
for molecule, content in (atoms | molecules).items():
    print(content)

{'geometry': 'F', 'open_shell': True}
{'geometry': 'H', 'open_shell': True}
{'De_exp': 163.35, 'De_calc': 161.04, 'geometry': 'F;F,F,1.4113', 'stoichiometry': 'FF'}
{'De_exp': 458.04, 'De_calc': 458.13, 'geometry': 'H;H,H,.7419', 'stoichiometry': 'HH'}


In [10]:
for method in methods:
    for basis in bases:
        for molecule, content in (atoms | molecules).items():
            p = proj(molecule,method,basis)
            root.children.append(p)
            p.write_input(
            f"""angstrom;geometry={{
            {content["geometry"]}
            }}
            basis = {basis}
            rhf
            {open_shell_methods[method] if content.get("open_shell") else closed_shell_methods[method]}
            """)

In [11]:
from multiprocessing.dummy import Pool
from operator import methodcaller
with Pool(processes=4) as pool:
    pool.map(methodcaller('run', backend=backend, wait=True),
             root.children, 1)
print('all completed', all_completed(root.children))
print('without errors', no_errors(root.children))

all completed False
without errors True


In [12]:
def get_energy(proj):
    return ([None]+proj.select('//jobstep/property[name=Energy].value') + proj.select('//jobstep/property[name=total energy].value'))[-1]

In [13]:
energies={}
for method in methods:
    energies[method]={}
    for basis in bases:
        energies[method][basis]={}
        for name, molecule in (atoms | molecules).items():
            energies[method][basis][name]=get_energy(proj(name,method,basis))
            print(method, basis, name,energies[method][basis][name])
        

B3LYP cc-pVDZ F -99.6909038089714
B3LYP cc-pVDZ H -0.497858658956189
B3LYP cc-pVDZ F2 -199.44486245998
B3LYP cc-pVDZ H2 -1.16656361368232
B3LYP cc-pVTZ F -99.7267255367631
B3LYP cc-pVTZ H -0.498765015439255
B3LYP cc-pVTZ F2 -199.517848816162
B3LYP cc-pVTZ H2 -1.17324207794776
HF cc-pVDZ F -99.3718619400978
HF cc-pVDZ H -0.499278403419583
HF cc-pVDZ F2 -198.685730306198
HF cc-pVDZ H2 -1.12871955893311
HF cc-pVTZ F -99.4009352721451
HF cc-pVTZ H -0.499809811301844
HF cc-pVTZ F2 -198.752127239944
HF cc-pVTZ H2 -1.13295027334918
MP2 cc-pVDZ F -99.5105296778471
MP2 cc-pVDZ H -0.499278403419583
MP2 cc-pVDZ F2 -199.079576895418
MP2 cc-pVDZ H2 -1.15510832068181
MP2 cc-pVTZ F -99.6001581300329
MP2 cc-pVTZ H -0.499809811301843
MP2 cc-pVTZ F2 -199.274531942485
MP2 cc-pVTZ H2 -1.16463331230473
CCSD cc-pVDZ F -99.526613482758
CCSD cc-pVDZ H -0.499278403419583
CCSD cc-pVDZ F2 -199.088444706371
CCSD cc-pVDZ H2 -1.16342731588449
CCSD cc-pVTZ F -99.6165681863372
CCSD cc-pVTZ H -0.499809811301843
CCSD c

In [14]:
energies

{'B3LYP': {'cc-pVDZ': {'F': -99.6909038089714,
   'H': -0.497858658956189,
   'F2': -199.44486245998,
   'H2': -1.16656361368232},
  'cc-pVTZ': {'F': -99.7267255367631,
   'H': -0.498765015439255,
   'F2': -199.517848816162,
   'H2': -1.17324207794776}},
 'HF': {'cc-pVDZ': {'F': -99.3718619400978,
   'H': -0.499278403419583,
   'F2': -198.685730306198,
   'H2': -1.12871955893311},
  'cc-pVTZ': {'F': -99.4009352721451,
   'H': -0.499809811301844,
   'F2': -198.752127239944,
   'H2': -1.13295027334918}},
 'MP2': {'cc-pVDZ': {'F': -99.5105296778471,
   'H': -0.499278403419583,
   'F2': -199.079576895418,
   'H2': -1.15510832068181},
  'cc-pVTZ': {'F': -99.6001581300329,
   'H': -0.499809811301843,
   'F2': -199.274531942485,
   'H2': -1.16463331230473}},
 'CCSD': {'cc-pVDZ': {'F': -99.526613482758,
   'H': -0.499278403419583,
   'F2': -199.088444706371,
   'H2': -1.16342731588449},
  'cc-pVTZ': {'F': -99.6165681863372,
   'H': -0.499809811301843,
   'F2': -199.278129196566,
   'H2': -1.17

In [15]:
De={}
for method in methods:
    De[method]={}
    for basis in bases:
        De[method][basis]={}
        for name, molecule in molecules.items():
            De[method][basis][name]=-energies[method][basis][name]
            for atom in molecule["stoichiometry"]:
                De[method][basis][name] += energies[method][basis][atom]
            De[method][basis][name] *= 2625.49963948
            print(method, basis, name,De[method][basis][name],molecule["De_exp"],molecule["De_calc"])

B3LYP cc-pVDZ F2 165.5504650361093 163.35 161.04
B3LYP cc-pVDZ H2 448.55688795047615 458.04 458.13
B3LYP cc-pVTZ F2 169.07625007363157 163.35 161.04
B3LYP cc-pVTZ H2 461.3319162326085 458.04 458.13
HF cc-pVDZ F2 -152.2621076228726 163.35 161.04
HF cc-pVDZ H2 341.7422586963748 458.04 458.13
HF cc-pVTZ F2 -130.6010276275133 163.35 161.04
HF cc-pVTZ H2 350.05957546392335 458.04 458.13
MP2 cc-pVDZ F2 153.63777944807956 163.35 161.04
MP2 cc-pVDZ H2 411.0259431539102 458.04 458.13
MP2 cc-pVTZ F2 194.85324743539266 163.35 161.04
MP2 cc-pVTZ H2 433.243382819356 458.04 458.13
CCSD cc-pVDZ F2 92.46416591814881 163.35 161.04
CCSD cc-pVDZ H2 432.8674620593826 458.04 458.13
CCSD cc-pVTZ F2 118.12864290655722 163.35 161.04
CCSD cc-pVTZ H2 453.46770013742037 458.04 458.13
CCSD(T) cc-pVDZ F2 111.36824191788547 163.35 161.04
CCSD(T) cc-pVDZ H2 432.8674620593826 458.04 458.13
CCSD(T) cc-pVTZ F2 146.16132905285303 163.35 161.04
CCSD(T) cc-pVTZ H2 453.46770013742037 458.04 458.13
